# Orbix - Demostración en Google Colab

Este notebook configura y demuestra las capacidades principales del sistema de navegación orbital Orbix utilizando algoritmos cuánticos.

## 1. Configuración del entorno

Primero, clonamos el repositorio e instalamos las dependencias necesarias:

In [ ]:
# Clonar el repositorio
!git clone https://github.com/nicolasleiva/Orbix.git
%cd Orbix

# Instalar dependencias principales
!pip install tensorflow qiskit pennylane matplotlib pandas numpy requests python-dotenv

## 2. Configuración para entorno Colab

Creamos un archivo de configuración temporal para el entorno de Colab:

In [ ]:
# Crear archivo de configuración para Colab
with open('src/colab_config.py', 'w') as f:
    f.write('''
# Configuración para entorno Colab
import os

# Configuración para simulación cuántica
QUANTUM_SIMULATOR_TYPE = "basic"
QUANTUM_SHOTS = 100
QUANTUM_NOISE_MODEL = "none"

# APIs (usando valores de demostración)
SPACEX_API_URL = "https://api.spacexdata.com/v4"
NOAA_TOKEN = "demo_token"
SPACE_TRACK_USERNAME = "demo_user"
SPACE_TRACK_PASSWORD = "demo_pass"
SSC_API_URL = "https://sscweb.gsfc.nasa.gov/WS/sscr/2"

# Configurar logger
import logging
logger = logging.getLogger("Orbix")
handler = logging.StreamHandler()
formatter = logging.Formatter(\'%(asctime)s - %(name)s - %(levelname)s - %(message)s\')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
''')
print("Created colab_config.py")

## 3. Modificar temporalmente la importación de configuración

In [ ]:
# Crear parche para importaciones
with open('src/colab_import_patch.py', 'w') as f:
    f.write('''
# Parche para importaciones en Colab
import sys
import os

# Añadir directorio actual al path
sys.path.insert(0, os.getcwd())

# Redireccionar importaciones de config a colab_config
import src.colab_config as config
sys.modules[\'src.config\'] = config
''')
print("Created colab_import_patch.py")

## 4. Demostración de predicción de trayectoria orbital

In [ ]:
# Aplicar parche de importación
import src.colab_import_patch

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import tensorflow as tf

# Importar componentes de Orbix
from src.quantum_trajectory import QuantumTrajectoryModel

# Inicializar el modelo de trayectoria cuántica
print("Inicializando modelo de trayectoria cuántica...")
model = QuantumTrajectoryModel()
print("Modelo inicializado correctamente")

# Crear datos de ejemplo para una trayectoria orbital
print("Generando datos de trayectoria de ejemplo...")
sample_trajectory = np.array([
    [7000 * np.cos(t), 7000 * np.sin(t), 1000 * np.sin(2*t)] 
    for t in np.linspace(0, 2*np.pi, 100)
])

# Visualizar trayectoria de ejemplo
plt.figure(figsize=(12, 8))
ax = plt.axes(projection='3d')
ax.plot3D(sample_trajectory[:, 0], sample_trajectory[:, 1], sample_trajectory[:, 2], 'blue')
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.set_title('Trayectoria orbital de ejemplo')
plt.show()

# Demostrar predicción de trayectoria
print("\nRealizando predicción de trayectoria con algoritmos cuánticos...")
# Añadir ruido para simular datos reales
noisy_data = sample_trajectory + np.random.normal(0, 100, sample_trajectory.shape)

# Preparar datos para el modelo
input_tensor = tf.convert_to_tensor(np.expand_dims(noisy_data, axis=0), dtype=tf.float32)

# Realizar predicción
try:
    predicted_trajectory = model(input_tensor).numpy()[0]
    
    # Visualizar comparación
    plt.figure(figsize=(12, 8))
    ax = plt.axes(projection='3d')
    ax.plot3D(noisy_data[:, 0], noisy_data[:, 1], noisy_data[:, 2], 'red', label='Datos con ruido')
    ax.plot3D(predicted_trajectory[:, 0], predicted_trajectory[:, 1], predicted_trajectory[:, 2], 'green', label='Predicción cuántica')
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title('Comparación: Datos con ruido vs Predicción cuántica')
    ax.legend()
    plt.show()
    print("Predicción completada exitosamente")
except Exception as e:
    print(f"Error en la predicción: {str(e)}")

## 5. Simulación de detección de colisiones

In [ ]:
from src.quantum_alerts import QuantumCollisionAlertSystem

# Inicializar sistema de alertas
print("Inicializando sistema de alertas de colisión...")
alert_system = QuantumCollisionAlertSystem()

# Crear datos de ejemplo para dos satélites
satellite1_trajectory = np.array([
    [7000 * np.cos(t), 7000 * np.sin(t), 1000 * np.sin(2*t)] 
    for t in np.linspace(0, 2*np.pi, 100)
])

satellite2_trajectory = np.array([
    [7200 * np.cos(t + 0.5), 7200 * np.sin(t + 0.5), 1200 * np.sin(2*t + 0.5)] 
    for t in np.linspace(0, 2*np.pi, 100)
])

# Visualizar ambas trayectorias
plt.figure(figsize=(12, 8))
ax = plt.axes(projection='3d')
ax.plot3D(satellite1_trajectory[:, 0], satellite1_trajectory[:, 1], satellite1_trajectory[:, 2], 'blue', label='Satélite 1')
ax.plot3D(satellite2_trajectory[:, 0], satellite2_trajectory[:, 1], satellite2_trajectory[:, 2], 'red', label='Satélite 2')
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.set_title('Trayectorias de dos satélites')
ax.legend()
plt.show()

# Simular detección de colisión
print("\nAnalizando riesgo de colisión...")
try:
    # Crear punto de aproximación cercana
    close_point_idx = 50
    satellite2_trajectory[close_point_idx] = satellite1_trajectory[close_point_idx] + np.array([10, 10, 10])
    
    # Convertir a formato esperado por el sistema de alertas
    sat1_data = {
        "satellite_id": "SAT1",
        "trajectory": satellite1_trajectory,
        "timestamp": [datetime.now() + timedelta(minutes=i*10) for i in range(len(satellite1_trajectory))]
    }
    
    sat2_data = {
        "satellite_id": "SAT2",
        "trajectory": satellite2_trajectory,
        "timestamp": [datetime.now() + timedelta(minutes=i*10) for i in range(len(satellite2_trajectory))]
    }
    
    # Detectar colisiones
    collision_results = alert_system.detect_potential_collisions([sat1_data, sat2_data])
    
    # Mostrar resultados
    print(f"Resultados de detección de colisión: {collision_results}")
    
    # Visualizar punto de aproximación cercana
    plt.figure(figsize=(12, 8))
    ax = plt.axes(projection='3d')
    ax.plot3D(satellite1_trajectory[:, 0], satellite1_trajectory[:, 1], satellite1_trajectory[:, 2], 'blue', label='Satélite 1')
    ax.plot3D(satellite2_trajectory[:, 0], satellite2_trajectory[:, 1], satellite2_trajectory[:, 2], 'red', label='Satélite 2')
    
    # Resaltar punto de aproximación
    ax.scatter(satellite1_trajectory[close_point_idx, 0], 
               satellite1_trajectory[close_point_idx, 1], 
               satellite1_trajectory[close_point_idx, 2], 
               color='green', s=100, label='Punto de aproximación')
    
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title('Detección de aproximación cercana entre satélites')
    ax.legend()
    plt.show()
    
except Exception as e:
    print(f"Error en la detección de colisiones: {str(e)}")